In [20]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import SparkSession
from pathlib import Path

In [21]:
conf = SparkConf().setAppName("spark-streaming-example")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=spark-streaming-example, master=yarn) created by __init__ at <ipython-input-2-2436b813b9cf>:2 

In [ ]:
! hadoop fs -put ../datasets/f1

In [ ]:
! hadoop fs -ls /user/root/f1

In [ ]:
! hadoop fs -mkdir /user/root/f1/drivers

In [ ]:
! hadoop fs -mkdir /user/root/output

In [ ]:
! hadoop fs -cp /user/root/f1/drivers.csv /user/root/f1/drivers/data.csv

In [ ]:
! hadoop fs -ls /user/root/f1/drivers

In [ ]:
schema = StructType([
    StructField("driverId", IntegerType(), True),
    StructField("driverRef", StringType(), True),
    StructField("number", IntegerType(), True),
    StructField("code", StringType(), True),
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True),
    StructField("dob", StringType(), True),
    StructField("nationality", StringType(), True),
    StructField("url", StringType(), True),
])

In [ ]:
sdf = spark.readStream.csv(
    path="/user/root/f1/drivers",
    schema=schema,
    sep=",",
    encoding="iso-8859-1",
    quote='"',
    header=True,
)

In [ ]:
def handle_data(batch_df, batch_id):
    batch_df.persist()
    batch_df.write.format(
        "csv"
    ).option(
        "path", "/user/root/output"
    ).mode(
        "write"
    ).save()
    batch_df.unpersist()

In [ ]:
sb = sdf.writeStream.outputMode("append").foreachBatch(handle_data).start()

In [ ]:
!echo '1002,"jp",99,"jp","Joao","Pinheiro","1988-11-17","Brazil","www.google.com"' | hdfs dfs -appendToFile - /user/root/f1/drivers/data.csv

In [ ]:
!echo '1003,"jp",99,"jp","Joao","Pinheiro","1988-11-17","Brazil","www.google.com"' | hdfs dfs -put - /user/root/f1/drivers/data1.csv

In [ ]:
!echo '1004,"jp",99,"jp","Joao","Pinheiro","1988-11-17","Brazil","www.google.com"' | hdfs dfs -put - /user/root/f1/drivers/data2.csv

In [ ]:
sb.isActive

In [ ]:
! hadoop fs -ls /user/root/output

In [ ]:
! hadoop fs -text /user/root/output/part-00000-21085dac-6505-443f-879a-ae876c8791d3-c000.csv

In [ ]:
! hadoop fs -text /user/root/f1/drivers/data.csv